## Automatic Attendance System

### Multiple check-ins and check-outs are not supported
### We track the state of each detected person to avoid multiple entries on the same day
### Unknown detections are not logged to the database

In [68]:
import sqlite3
import cv2
from ultralytics import YOLO
import time
import threading
from datetime import datetime, timedelta

In [69]:
model = YOLO(r"C:\Users\Hamza\Desktop\Deep Learning\Assignment 3\best.pt")

In [70]:
conn = sqlite3.connect('attendance25.db')
conn.execute('''CREATE TABLE IF NOT EXISTS attendance_records
             (id INTEGER PRIMARY KEY,
              name TEXT NOT NULL,
              date TEXT NOT NULL,
              check_in_time INTEGER,
              check_out_time INTEGER,
              working_time INTEGER)''')
conn.commit()

In [71]:
conn

In [72]:
day_counter = 0
custom_time_counter = 0

In [73]:
# Global flags for thread control
day_counter_running = True
time_counter_running = True

In [74]:
# Base date for the simulation set to today's date
base_date = datetime.now().date()

In [75]:
# Function to get simulated date based on day_counter
def get_simulated_date(day_counter):
    return base_date + timedelta(days=day_counter)

In [76]:
def increment_day_counter():
    global day_counter, day_counter_running
    while day_counter_running:
        time.sleep(8)
        day_counter += 1

In [77]:
def increment_custom_time_counter():
    global custom_time_counter, time_counter_running
    while time_counter_running:
        time.sleep(1)
        custom_time_counter += 1

In [78]:
# Start the day and custom time counter threads
day_counter_thread = threading.Thread(target=increment_day_counter, daemon=True)
day_counter_thread.start()
custom_time_counter_thread = threading.Thread(target=increment_custom_time_counter, daemon=True)
custom_time_counter_thread.start()

In [79]:
# Function to get current time offset within the day
def get_time_offset():
    return custom_time_counter % 8

In [80]:
# Function to update attendance
def update_attendance(name):
    global day_counter
    current_time_offset = get_time_offset()
    simulated_date = get_simulated_date(day_counter)
    date_str = simulated_date.strftime("%Y-%m-%d")

    cursor = conn.execute("SELECT id, check_in_time, check_out_time FROM attendance_records WHERE name=? AND date=?", (name, date_str))
    record = cursor.fetchone()

    if record:
        if record[2] is None:
            working_time = min(30, current_time_offset - record[1])
            conn.execute("UPDATE attendance_records SET check_out_time=?, working_time=? WHERE id=?", (current_time_offset, working_time, record[0]))
            print(f"Checked out {name} with working time {working_time} seconds.")
    else:
        conn.execute("INSERT INTO attendance_records (name, date, check_in_time) VALUES (?, ?, ?)", (name, date_str, current_time_offset))
        print(f"Checked in {name} at time offset {current_time_offset}.")
    conn.commit()

In [81]:
# Function to stop threads
def stop_threads():
    global day_counter_running, time_counter_running
    day_counter_running = False
    time_counter_running = False

In [82]:
# Start video capture
cap = cv2.VideoCapture(0)

# Tracking the state of each detected person 
person_states = {}

# Process video frames
while cap.isOpened():
    success, frame = cap.read()
    if success:
        results = model(frame)

        detected_names = set()
        for result in results:
            boxes = result.boxes.cpu().numpy()
            for box in boxes:
                person_name = result.names[int(box.cls[0])]
                detected_names.add(person_name)

                if person_states.get(person_name, 'not_detected') == 'not_detected':
                    update_attendance(person_name)
                    person_states[person_name] = 'detected'

        for name in person_states.keys():
            if name not in detected_names:
                person_states[name] = 'not_detected'

        cv2.imshow("YOLOv8 Inference", frame)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    else:
        break

# Release resources and stop threads
cap.release()
cv2.destroyAllWindows()
stop_threads()
day_counter_thread.join()
custom_time_counter_thread.join()
conn.close()


0: 480x640 1 Me, 1 My friend, 164.1ms
Speed: 5.7ms preprocess, 164.1ms inference, 0.9ms postprocess per image at shape (1, 3, 480, 640)
Checked in My friend at time offset 0.
Checked in Me at time offset 0.

0: 480x640 1 My friend, 173.6ms
Speed: 1.9ms preprocess, 173.6ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 My friend, 149.1ms
Speed: 2.0ms preprocess, 149.1ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 My friend, 143.7ms
Speed: 3.0ms preprocess, 143.7ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 My friend, 141.0ms
Speed: 2.0ms preprocess, 141.0ms inference, 1.1ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 My friend, 143.4ms
Speed: 2.2ms preprocess, 143.4ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 My friend, 135.7ms
Speed: 2.4ms preprocess, 135.7ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)

0: 48

In [83]:
# View database records
# Connect to the database
conn = sqlite3.connect('attendance25.db')

# Create a cursor object
cursor = conn.cursor()

# Fetch all records
cursor.execute("SELECT * FROM attendance_records")
records = cursor.fetchall()

# Print records
for record in records:
    print(record)

# Close the connection
conn.close()


(1, 'My friend', '2024-01-16', 0, 2, 2)
(2, 'Me', '2024-01-16', 0, None, None)
(3, 'My friend', '2024-01-17', 1, 4, 3)
(4, 'My friend', '2024-01-18', 0, 3, 3)
(5, 'My friend', '2024-01-19', 2, 5, 3)
(6, 'My friend', '2024-01-20', 0, 2, 2)
(7, 'My friend', '2024-01-21', 1, None, None)
